In [55]:
import pandas as pd
import requests

In [56]:
class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)
        
    def __str__(self):
        return str(self.lat) + ',' + str(self.lon)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon

    def __hash__(self):
        return hash((self.lat, self.lon))


In [57]:
id_coord = pd.read_csv('population_lolland_4500_0.csv', header=None)

In [58]:
id_coord

,0,1,2
0,0,11.138917,54.832973
1,1,11.089274,54.876195
2,2,11.139552,54.833045
3,3,11.144421,54.820618
4,4,11.142571,54.830062
...,...,...,...
231,231,11.124462,54.898505
232,232,11.010379,54.775875
233,233,11.173146,54.833725
234,234,11.139268,54.827073


In [59]:
id_coord['coord'] = id_coord.apply(lambda x: Coord(lon=x[1], lat=x[2]), axis=1)

In [60]:
def _osrm_tdm_request(coords):
    url_coords = ';'.join([str(coord.lon) + ',' + str(coord.lat) for coord in coords])
    url_server = 'http://0.0.0.0:5000/table/v1/driving/'
    url_options = 'fallback_speed=9999999999&annotations=duration,distance'
    url_full = '{}{}?{}'.format(url_server, url_coords, url_options)
    resp = requests.get(url=url_full)

    jresp = resp.json()
    if jresp.get('code') != 'Ok':
        log.error(jresp.get('code'))
        log.error(jresp.get('message'))
        resp.raise_for_status()

    return jresp.get('durations'), jresp.get('distances')

In [61]:
tdm = _osrm_tdm_request(id_coord.coord)

In [62]:
df = []
durations = tdm[0]
distances = tdm[1]
for source, duration_row, distance_row in zip(range(0, len(durations)+1), durations, distances):
    for destination, duration, distance in zip(range(0, len(durations)+1), duration_row, distance_row):
        df.append((source,
                  destination,
                  duration,
                  distance))

In [63]:
pd.DataFrame(df).to_csv('tdm_lolland.csv', header=None, index=None)

In [ ]:
pd.DataFrame(df).to_feather('tdm.feater')